# Differentiable bitonic sort

[Bitonic sorts](https://en.wikipedia.org/wiki/Bitonic_sorter) allow creation of sorting networks with a sequence of fixed conditional swapping operations executed in parallel. A sorting network implements  a map from $\mathbb{R}^n \rightarrow \mathbb{R}^n$, where $n=2^k$ (sorting networks for non-power-of-2 sizes are possible but not trickier).

<img src="BitonicSort1.svg.png">

*[Image: from Wikipedia, by user Bitonic, CC0](https://en.wikipedia.org/wiki/Bitonic_sorter#/media/File:BitonicSort1.svg)*

The sorting network for $n=2^k$ elements has $\frac{k(k-1)}{2}$ "layers" where parallel compare-and-swap operations are used to rearrange a $k$ element vector into sorted order.

### Differentiable compare-and-swap

If we define the `softmax(a,b)` function (not the traditional "softmax" used for classification!) as the continuous approximation to the `max(a,b)` function:

$$\text{softmax}(a,b) = \log(e^a + e^b) \approx \max(a,b).$$

We can then fairly obviously write `softmin(a,b)` as:

$$\text{softmin}(a,b) = -\log(e^{-a} + e^{-b}) \approx \min(a,b).$$

These functions obviously aren't equal to max and min, but are relatively close, and differentiable. Note that we now have a differentiable compare-and-swap operation:

$$\text{high} = \text{softmax}(a,b), \text{low} = \text{softmin}(a,b), \text{where } \text{low}\leq \text{high}$$

## Differentiable sorting

For each layer in the sorting network, we can split all of the pairwise comparison-and-swaps into left-hand and right-hand sides which can be done simultaneously. We can any write function that selects the relevant elements of the vector as a multiply with a binary matrix.

For each layer, we can derive two binary matrices $L \in \mathbb{R}^{n \times \frac{n}{2}}$ and $R \in \mathbb{R}^{n \times \frac{n}{2}}$ which select the elements to be compared for the left and right hands respectively. This will result in the comparison between two $\frac{k}{2}$ length vectors. We can also derive two matrices $L' \in \mathbb{R}^{\frac{n}{2} \times n}$ and $R' \in \mathbb{R}^{\frac{n}{2} \times n}$ which put the results of the compare-and-swap operation back into the right positions.

Then, each layer $i$ of the sorting process is just:
$${\bf x}_{i+1} = L'_i[\text{softmin}(L_i{\bf x_i}, R_i{\bf x_i})] + R'_i[\text{softmax}(L_i{\bf x_i}, R_i{\bf x_i})]$$
$$ = L'_i\left(-\log\left(e^{-L_i{\bf x}_i} + e^{-R_i{\bf x}_i}\right)\right) +  R'_i\left(\log\left(e^{L_i{\bf x}_i} + e^{R_i{\bf x}_i}\right)\right)$$
which is clearly differentiable (though not very numerically stable -- the usable range of elements $x$ is quite limited in single float precision).

All that remains is to compute the matrices $L_i, R_i, L'_i, R'_i$ for each of the layers of the network. 

This process is excessively computation heavy, but easy to compute. We could also simplify this into two matrix multiplies, at the cost of a vector split and join in the middle (see the `woven` form later in this text). 

## Example

To sort four elements, we have a network like:

    0  1  2  3  
    ┕>>┙  │  │  
    │  │  ┕<<┙  
    ┕>>>>>┙  │  
    │  │  │  │  
    ┕>>┙  │  │  
    │  │  ┕>>┙  
    
This is equivalent to: 

    x[0], x[1] = cswap(x[0], x[1])
    x[3], x[2] = cswap(x[2], x[3])
    x[0], x[2] = cswap(x[0], x[2])
    x[0], x[1] = cswap(x[0], x[1])
    x[2], x[3] = cswap(x[2], x[3])
    
where `cswap(a,b) = (min(a,b), max(a,b))`

Replacing the indexing with matrix multiplies and `cswap` with a `softcswap = (softmin(a,b), softmax(a,b))` we then have the differentiable form.



# Test functions

In [1]:
from bitonic_tests import bitonic_network, pretty_bitonic_network

# this should match the diagram at the top of the notebook
bitonic_network(16)

 0>1	 2<3	 4>5	 6<7	 8>9	10<11	12>13	14<15	
----------------------------------------------------------------
 0>2	 1>3	 4<6	 5<7	 8>10	 9>11	12<14	13<15	
 0>1	 2>3	 4<5	 6<7	 8>9	10>11	12<13	14<15	
----------------------------------------------------------------
 0>4	 1>5	 2>6	 3>7	 8<12	 9<13	10<14	11<15	
 0>2	 1>3	 4>6	 5>7	 8<10	 9<11	12<14	13<15	
 0>1	 2>3	 4>5	 6>7	 8<9	10<11	12<13	14<15	
----------------------------------------------------------------
 0>8	 1>9	 2>10	 3>11	 4>12	 5>13	 6>14	 7>15	
 0>4	 1>5	 2>6	 3>7	 8>12	 9>13	10>14	11>15	
 0>2	 1>3	 4>6	 5>7	 8>10	 9>11	12>14	13>15	
 0>1	 2>3	 4>5	 6>7	 8>9	10>11	12>13	14>15	
----------------------------------------------------------------


In [12]:
pretty_bitonic_network(16)

0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 
+>>+  │  │  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  +<<+  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  +>>+  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  │  │  +<<+  │  │  │  │  │  │  │  │  
│  │  │  │  │  │  │  │  +>>+  │  │  │  │  │  │  
│  │  │  │  │  │  │  │  │  │  +<<+  │  │  │  │  
│  │  │  │  │  │  │  │  │  │  │  │  +>>+  │  │  
│  │  │  │  │  │  │  │  │  │  │  │  │  │  +<<+  
+>>>>>+  │  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  +<<<<<+  │  │  │  │  │  │  │  │  │  
│  │  │  │  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  │  │  │  │  +>>>>>+  │  │  │  │  │  
│  │  │  │  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  │  │  │  │  │  │  │  │  +<<<<<+  │  
│  │  │  │  │  │  │  │  │  │  │  │  │  │  │  │  
+>>+  │  │  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  +>>+  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  +<<+  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  │  │  +<

# Vectorised functions

## Testing

In [24]:
# Test sorting
import numpy as np

from differentiable_sorting import bitonic_matrices, diff_bisort, diff_rank, softmax, softmin, softcswap, bisort
matrices = bitonic_matrices(8)

for i in range(10):
    # these should all be in sorted order
    test = np.random.randint(0, 200, 8)
    print(bisort(matrices, test))    

[ 10.  52.  56.  60.  69.  72.  88. 161.]
[ 28.  82.  96. 112. 115. 129. 157. 159.]
[ 25.  40.  75.  89. 134. 148. 162. 169.]
[ 23.  51.  55.  79.  84. 101. 127. 180.]
[  1.  12.  53. 124. 143. 149. 178. 180.]
[ 44.  56.  79. 111. 129. 143. 152. 194.]
[ 53. 149. 155. 177. 182. 184. 192. 195.]
[ 47.  62.  86.  91. 102. 169. 186. 198.]
[  6.  74.  90. 126. 140. 142. 155. 163.]
[  6.  29.  31. 133. 139. 178. 180. 184.]


In [25]:
for i in range(1, 11):
    k = 2**i
    matrices = bitonic_matrices(k)
    print(f"Testing sorting for {k} elements")
    for j in range(100):
        test = np.random.randint(0, 200, k)
        assert (np.allclose(bisort(matrices, test), np.sort(test)))

Testing sorting for 2 elements
Testing sorting for 4 elements
Testing sorting for 8 elements
Testing sorting for 16 elements
Testing sorting for 32 elements
Testing sorting for 64 elements
Testing sorting for 128 elements
Testing sorting for 256 elements
Testing sorting for 512 elements
Testing sorting for 1024 elements


## Differentiable sorting test

In [26]:
# Differentiable sorting 
np.set_printoptions(precision=2)
matrices = bitonic_matrices(8) 
def neat_vec(n):
    return "\t".join([f"{x:.2f}" for x in n])

for i in range(10):
    test = np.random.randint(-200,200,8)
    print("Differentiable", neat_vec(diff_bisort(matrices, test)))
    print("Exact sorting ", neat_vec(bisort(matrices, test)))
    print()

Differentiable -160.05	-158.64	-157.30	-124.00	-115.00	-71.00	-45.01	-39.99
Exact sorting  -159.00	-159.00	-158.00	-124.00	-115.00	-71.00	-45.00	-40.00

Differentiable -112.62	-110.38	-100.13	-96.87	-65.62	-63.38	36.00	107.00
Exact sorting  -112.00	-111.00	-100.00	-97.00	-65.00	-64.00	36.00	107.00

Differentiable -64.00	-6.00	18.00	30.00	74.00	98.00	108.00	175.00
Exact sorting  -64.00	-6.00	18.00	30.00	74.00	98.00	108.00	175.00

Differentiable -167.00	-86.00	-21.00	-12.00	21.00	141.00	152.00	170.00
Exact sorting  -167.00	-86.00	-21.00	-12.00	21.00	141.00	152.00	170.00

Differentiable -197.00	-163.00	-46.00	74.00	100.00	107.00	145.00	197.00
Exact sorting  -197.00	-163.00	-46.00	74.00	100.00	107.00	145.00	197.00

Differentiable -157.00	-96.00	-74.00	-54.00	65.00	108.95	112.05	143.00
Exact sorting  -157.00	-96.00	-74.00	-54.00	65.00	109.00	112.00	143.00

Differentiable -128.00	-71.02	-66.98	-50.00	-29.00	98.00	153.00	172.00
Exact sorting  -128.00	-71.00	-67.00	-50.00	-29.00	98.00	153.00	1

# Relaxed sorting
We can define a slighly modified function which interpolates between `softmax(a,b)` and `mean(a,b)`. The result is a sorting function that can be relaxed from sorting to averaging.

In [27]:
from differentiable_sorting import diff_bisort_smooth
# Differentiable smoothed sorting 
test = np.random.randint(-200,200,8)
print(f"Mean {np.mean(test):.2f}")
print()
print("Exact sorting       ", neat_vec(bisort(matrices, test)))
for smooth in np.linspace(0, 1, 8):    
    print(f"Diff. smooth[{smooth:.2f}]  ", neat_vec(diff_bisort_smooth(matrices, test, smooth)))
        

Mean -105.62

Exact sorting        -187.00	-179.00	-173.00	-171.00	-166.00	-80.00	-46.00	157.00
Diff. smooth[0.00]   -187.00	-179.00	-173.22	-170.78	-165.99	-80.00	-46.00	157.00
Diff. smooth[0.14]   -159.32	-157.79	-151.89	-150.12	-122.83	-86.27	-68.29	51.51
Diff. smooth[0.29]   -143.87	-142.95	-136.29	-128.65	-101.87	-97.44	-76.67	-17.26
Diff. smooth[0.43]   -131.19	-130.47	-125.18	-118.16	-100.76	-95.40	-84.55	-59.29
Diff. smooth[0.57]   -122.58	-122.08	-119.86	-118.96	-96.54	-94.20	-88.99	-81.79
Diff. smooth[0.71]   -115.94	-115.62	-115.48	-115.16	-97.40	-96.84	-94.73	-93.84
Diff. smooth[0.86]   -110.40	-110.25	-110.23	-110.08	-101.22	-101.07	-100.95	-100.80
Diff. smooth[1.00]   -105.62	-105.62	-105.62	-105.62	-105.62	-105.62	-105.62	-105.62


## Woven form
We can "weave" the four matrices into two matrices for fewer multiplies at the cost of having to split and join the matrices at each layer.

In [28]:
from differentiable_sorting import diff_bisort_weave, bisort_woven_matrices

woven_matrices = bisort_woven_matrices(8)

print("Exact sorting       ", neat_vec(bisort(matrices, test)))
print(f"Diff. (std.)       ", neat_vec(diff_bisort(matrices, test)))
print(f"Diff. (woven)      ", neat_vec(diff_bisort_weave(woven_matrices, test)))
        

Exact sorting        -187.00	-179.00	-173.00	-171.00	-166.00	-80.00	-46.00	157.00
Diff. (std.)        -187.00	-179.00	-173.22	-170.78	-165.99	-80.00	-46.00	157.00
Diff. (woven)       -187.00	-179.00	-173.22	-170.78	-165.99	-80.00	-46.00	157.00


## Differentiable ranking
We can use a differentiable similarity measure between the input and output of the vector, e.g. an RBF kernel. We can use this to generate a normalised similarity matrix and apply this to a vector `[1, 2, 3, ..., n]`. This gives a differentiable ranking function.

As `sigma` gets larger, the result converges to giving all values the mean rank; as it goes to zero the result converges to the true rank.

In [29]:
from differentiable_sorting import order_matrix, diff_rank

In [30]:
matrices = bitonic_matrices(8)
test = np.random.randint(0, 200, 8)
sortd = diff_bisort(matrices, test)

In [31]:
print("Smoothed ranks")
for sigma in [0.1, 1, 10, 100, 1000]:
    similarity = order_matrix(test, sortd, sigma=sigma)    
    ranks = np.arange(len(test))    
    print(f"sigma={sigma:7.1f}  |", neat_vec(similarity @ ranks))

Smoothed ranks
sigma=    0.1  | 6.00	0.00	3.00	2.00	7.00	2.00	4.00	5.00
sigma=    1.0  | 6.00	0.00	2.36	1.76	7.00	1.76	4.00	5.00
sigma=   10.0  | 6.02	0.80	2.25	2.19	6.97	2.19	2.72	5.00
sigma=  100.0  | 4.34	2.69	2.84	2.83	4.70	2.83	2.92	3.86
sigma= 1000.0  | 3.51	3.49	3.49	3.49	3.51	3.49	3.49	3.50


In [32]:
def diff_rank(matrices, x, sigma=0.1):
    """Return the smoothed, differentiable ranking of each element of x. Sigma
    specifies the smoothing of the ranking. """
    sortd = diff_bisort(matrices, x)
    return order_matrix(x, sortd, sigma=sigma) @ np.arange(len(x))

In [33]:
x = [5.0, -1.0, 9.5, 13.2, 16.2, 20.5, 42.0, 18.0]
np.set_printoptions(suppress=True)
print(neat_vec((diff_rank(matrices, x, sigma=0.1))))

1.00	0.00	2.00	3.00	4.00	6.00	7.00	5.00


# PyTorch example
We can verify that this is both parallelisable on the GPU and fully differentiable.

In [1]:
import torch
import numpy as np
from torch.autograd import Variable
import torch.nn.functional as F
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', device)

Device: cpu


In [5]:
from differentiable_sorting_torch import diff_bisort, bitonic_matrices, diff_rank
matrices = bitonic_matrices(16)
torch_matrices = [[torch.from_numpy(matrix).float().to(device) for matrix in matrix_set] for matrix_set in matrices]


In [6]:
test_input = np.random.normal(0, 5, 16)
var_test_input = Variable(torch.from_numpy(test_input).float().to(device),
                          requires_grad=True)

result = diff_bisort(torch_matrices, var_test_input)

# compute the Jacobian of the sorting function, to show we can differentiate through the
# sorting function
jac = []
for i in range(len(result)):
    jac.append(
        torch.autograd.grad(result[i], var_test_input, retain_graph=True)[0])

# 16 x 16 jacobian of the sorting matrix
print(torch.stack(jac))

tensor([[2.5131e-03, 5.2202e-05, 6.7652e-05, 1.5480e-02, 1.4476e-04, 5.9317e-05,
         5.3141e-03, 7.3452e-04, 5.8550e-04, 1.5430e-04, 9.6412e-06, 6.1721e-07,
         9.7262e-01, 2.1492e-03, 5.2062e-06, 1.0513e-04],
        [9.0604e-02, 1.8423e-03, 2.3955e-03, 6.6582e-01, 4.7377e-03, 1.9671e-03,
         1.5011e-01, 2.2408e-02, 1.0846e-02, 2.4933e-03, 1.6341e-04, 1.0141e-05,
         2.1643e-02, 2.3574e-02, 6.5396e-05, 1.3220e-03],
        [1.7588e-01, 3.2152e-03, 4.0942e-03, 1.5777e-01, 1.0447e-02, 4.2611e-03,
         4.0763e-01, 5.2987e-02, 5.9876e-02, 1.4155e-02, 8.1245e-04, 4.9536e-05,
         2.8100e-03, 9.8990e-02, 3.7328e-04, 6.6456e-03],
        [2.3339e-01, 4.8711e-03, 6.2478e-03, 1.0802e-01, 1.4679e-02, 5.5586e-03,
         2.1918e-01, 6.2734e-02, 1.0191e-01, 2.0118e-02, 1.3029e-03, 7.9378e-05,
         1.5999e-03, 2.1119e-01, 4.7283e-04, 8.6520e-03],
        [1.6945e-01, 1.7494e-02, 2.2230e-02, 1.4854e-02, 4.5408e-02, 2.0010e-02,
         6.0525e-02, 1.2786e-01, 1.8481

In [9]:
result = diff_rank(torch_matrices, var_test_input)
print(result)


-1


RuntimeError: Device index must not be negative